In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import os, sys, json, pickle
sys.path.append('../scripts/')
import land_cover_analysis as lca
import land_cover_visualisation as lcv
import land_cover_models as lcm
import loadpaths 

path_dict = loadpaths.loadpaths()  # load paths (see content/README_datapaths.md)


## Loading existing models
If you want to use the CNN models from our paper, you can download these [here](https://drive.google.com/drive/folders/1nEnIWDvWcLVzSE6yViv93I4klY2WzdDo?usp=sharing).

In the following, we will load these (also applicable to other models).

There are two ways of loading models: loading the full model (`.data`), or loading the resnet weights only (`.pth`). The `lcm.load_model_auto()` function does this automatically, by recognising the file extension. To load the full models provided, you need an older version of torch/lightning (see `envs/geo_exactbuilds.txt`). The weights-only models can also be loaded with latest torch/lightning, and are equivalent.

In [29]:
folder_model = '/home/thijs/Google Drive/peak district/models/2024-03-22/saved_cnn_models'
# file_name = 'main_LCU_2023-04-24-1259_FULL.data'
file_name = 'main_LCU_2023-04-24-1259.pth'

In [30]:
LCU = lcm.load_model_auto(folder=folder_model, filename=file_name)

cross_entropy loss is used.


## Loading the train/test patch data set
To train and test the model, you can use the data set of image patches (= 512x 512 pixels) from our paper [available here](https://cord.cranfield.ac.uk/articles/dataset/Very_high_resolution_aerial_photography_and_annotated_land_cover_data_of_the_Peak_District_National_Park/24221314). These can be loaded as follows:

In [22]:
mapping_dicts_list = [
        #   '../content/label_mapping_dicts/label_mapping_dict__all_relevant_subclasses__2023-04-20-1540.pkl'
        '../content/label_mapping_dicts/label_mapping_dict__C_subclasses_only__2023-04-20-1540.pkl',
        '../content/label_mapping_dicts/label_mapping_dict__D_subclasses_only__2023-04-20-1540.pkl',
        '../content/label_mapping_dicts/label_mapping_dict__E_subclasses_and_F3d_only__2023-04-20-1541.pkl',
        '../content/label_mapping_dicts/label_mapping_dict__main_categories_F3inDE_noFGH__2023-04-21-1315.pkl'
                         ]

In [24]:
for path_mapping_dict in mapping_dicts_list:
    tmp_path_dict = pickle.load(open(path_mapping_dict, 'rb'))
    n_classes = len(tmp_path_dict['dict_new_names'])
    print(path_mapping_dict, n_classes)
    

../content/label_mapping_dicts/label_mapping_dict__C_subclasses_only__2023-04-20-1540.pkl 5
../content/label_mapping_dicts/label_mapping_dict__D_subclasses_only__2023-04-20-1540.pkl 7
../content/label_mapping_dicts/label_mapping_dict__E_subclasses_and_F3d_only__2023-04-20-1541.pkl 5
../content/label_mapping_dicts/label_mapping_dict__main_categories_F3inDE_noFGH__2023-04-21-1315.pkl 4
